# Confounding an interaction
**By: Edvin Maid** \
**Date: 07 June 2023**

The idea of this study is to answer the following question:

<div style="background-color: #d4d3d3; padding: 10px; border-radius: 5px;">
    <p>Researchers have found that underweight newborns whose mothers smoked during pregnancy are less likely to die than underweight newborns whose mothers did not smoke. Granted that smoking during pregnancy increases the risk of infant death, provide a hypothesis that could explain this finding.</p>
</div>



In this study we plan to propose a data generating story in which the data is confounded. To get the effect envisioned by the question we must build such a randomly generated dataset such that when we quantify the effect of smoking is lesser than of the interaction term, while achieving low p-values of parameters

In [1]:
# Import needed libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

Here we define a function that generates the data where for given distributions of variables (making them independant) we generate a target (outcome) variable as a General Linear Model (GLM) in our case a binomial model.

In [2]:
def generate_data(coefficients, distributions, target, target_type, number_of_datapoints):
    """
    Randomly generates a pandas dataframe containing rows with given distributions and a target column
    the value of the target column is generated as a linear model using the previously generated covariates

    Args:
        coefficients (dict): A dictionary containing the coefficients for each feature.
        distributions (dict): A dictionary specifying the distribution parameters for each feature.
        target (str): The name of the target variable.
        target_type (str): The type of the target variable ("binary" or other).
        number_of_datapoints (int): The number of datapoints to generate.

    Returns:
        pandas.DataFrame: A DataFrame containing the generated data.

    Raises:
        None

    """
    data = {}
    
    # Generate data for each feature
    for feature, dist_params in distributions.items():
        distribution, *params = dist_params
        
        if distribution == "bernouli":
            data[feature] = np.random.binomial(1, params[0], number_of_datapoints)
        elif distribution == "gaussian":
            data[feature] = np.random.normal(*params, number_of_datapoints)
        elif distribution == 'constant':
            data[feature] = np.full(number_of_datapoints, params[0])
    
    # Generate target variable
    target_data = np.zeros(number_of_datapoints)
    data[target] = target_data
    
    # Create dataframe from generated data
    df = pd.DataFrame(data)
    
    # Apply coefficients to features
    for feature, coefficient in coefficients.items():
        if ":" in feature:
            sub_features = feature.split(":")
            df[target] += df[sub_features[0]] * df[sub_features[1]] * coefficient
        else:
            df[target] += df[feature]*coefficient
    
    df[target] = df[target].apply(lambda x: 1 / (1 + np.exp(-x)))
    
    if target_type == "binary":
        target_data = np.random.binomial(1, df[target], number_of_datapoints)
        df[target] = target_data
    
    return df


## How it's done
To cause the interaction term of an assumed model in which an unobserved confounder is that some countries are underdeveloped and other are developed. We need to construct two populations. In one the death rate will be high just because it is not developed and the medical institutions and general knowledge is very bad. However, the rate of smoking in the underdeveloped country must be a lot lower, but the rate of underweight newborns is high. 

The values of all coefficients given below are arbitrarily chosen and have no relation with any real world scenarios.

In [3]:
# The GLM coefficients of the data generating process are given below
coefficients = {"smoking":0.2, "underweight": 0.2, "smoking:underweight": 0.1, 'underdeveloped':10}

# Construct two populations with different distributions to emphisize a correlation with whether the country is
# developed or not.

distributions_underdeveloped = {"smoking": ("bernouli", 0.03), "underweight": ("bernouli", 0.9), 'underdeveloped':('constant', 1)}
data_underdeveloped = generate_data(coefficients, distributions_underdeveloped, target="died_under_age_of_2", target_type="binary", number_of_datapoints=10000)

distributions_developed = {"smoking": ("bernouli", 0.5), "underweight": ("bernouli", 0.05), 'underdeveloped':('constant', 0)}
data_developed = generate_data(coefficients, distributions_developed, target="died_under_age_of_2", target_type="binary", number_of_datapoints=100000)

# constuct dataset
data = pd.concat([data_underdeveloped, data_developed])
data.head()

,smoking,underweight,underdeveloped,died_under_age_of_2
0,0,1,1,1
1,0,1,1,1
2,0,1,1,1
3,0,1,1,1
4,0,1,1,1


## Logistic regression
Here we apply logistic regression with a model that does oblivious of the fact that the data comes from two populations at all. With this we try an fit only the smoking and underweight variables as well as its interaction term. However, because of the unobserved confounder we will get a negative interaction term.

In [4]:
formula = 'died_under_age_of_2 ~ smoking * underweight'
# Fit the GLM with Bernoulli distribution
model = sm.GLM.from_formula(formula, family=sm.families.Binomial(), data=data)
results = model.fit()

We should be able to re run this expriment as many times as we want and most of the time get a good P value <0.05

In [5]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                  Generalized Linear Model Regression Results                  
===============================================================================
Dep. Variable:     died_under_age_of_2   No. Observations:               110000
Model:                             GLM   Df Residuals:                   109996
Model Family:                 Binomial   Df Model:                            3
Link Function:                   Logit   Scale:                          1.0000
Method:                           IRLS   Log-Likelihood:                -71815.
Date:                 Wed, 07 Jun 2023   Deviance:                   1.4363e+05
Time:                         17:04:34   Pearson chi2:                 1.10e+05
No. Iterations:                      5   Pseudo R-squ. (CS):            0.05916
Covariance Type:             nonrobust                                         
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0372      0.009      4.100      0.000       0.019       0.055
smoking                 0.1609      0.013     12.420      0.000       0.135       0.186
underweight             2.1205      0.032     65.857      0.000       2.057       2.184
smoking:underweight    -1.7078      0.052    -33.140      0.000      -1.809      -1.607
=======================================================================================
"""